# California 2Monthly Prediction

## Imports and Functions

In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.gaussian_process.kernels import RBF, WhiteKernel
from sklearn.gaussian_process import GaussianProcessRegressor

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, Dense, Dropout, GRU, Conv1D, Flatten, Reshape

from sklearn.svm import SVR
import os
import numpy as np
from xgboost import XGBRegressor
import sys

In [2]:
# Add the folder to the Python path

os.chdir("../")
# change working directory to project's root path
print(os.getcwd())

FIRST_YEAR= 1972
FREQUENCY= "2monthly"

c:\Users\marti\Desktop\data\hw_extra


In [3]:
folder_path = os.path.abspath("functions/") #INPUT_PATH)#'path_to_your_folder')  # Replace with the actual folder path
sys.path.insert(0, folder_path)

from Predictions import (
    PredictionExperiment,
    PredictionModel
)

c:\Users\marti\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [4]:
region="california"
metadata = pd.read_csv(f"data/new_features_t2m/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(inplace=True, drop=True)
display(metadata)

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,dc3bd026,predictor_dc3bd026_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,False
1,dc3bd026,predictor_dc3bd026_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,False
2,dc3bd026,predictor_dc3bd026_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,False
3,dc3bd026,predictor_dc3bd026_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,False
4,dc3bd026,predictor_dc3bd026_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,f8698b6f,predictor_f8698b6f_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,f8698b6f,predictor_f8698b6f_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,f8698b6f,predictor_f8698b6f_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,f8698b6f,predictor_f8698b6f_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [5]:
# kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
# regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
#                 XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
# name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
# indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
# id_experiments = metadata["id"].unique()
# for id in id_experiments:
#     data = {i: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
#     experiment_1 = PredictionExperiment(data, indices_of_interest, regressors, name_regressors, 5, id)
#     experiment_1.execute_experiment()
#     experiment_1.get_metrics("r2", "prediction", show=False)
#     experiment_1.get_metrics("mape", "prediction", show=False)
#     experiment_1.get_metrics("r2", stage="training", show=False)
#     experiment_1.get_metrics("mape", stage="training", show=False)
#     experiment_1.get_metrics("r2", stage="CV", show=False)
#     experiment_1.get_metrics("mape", stage="CV", show=False)
#     #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
#     experiment_1.save_results(f"data/{region}_results/results.csv")

In [6]:
k=0
id_experiments = metadata["id"].unique()
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
print(len(id_experiments))
for id in id_experiments:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    # rnn_model = Sequential([
    # SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # lstm_model = Sequential([
    # LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # Stacked LSTM
    stacked_lstm = Sequential([
        LSTM(16, activation="tanh", return_sequences=True, input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        LSTM(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    cnn_rnn_model = Sequential([
        Conv1D(16, kernel_size=1, activation="relu", input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        Reshape((1, 16)),  # Back to time dimension
        SimpleRNN(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [stacked_lstm, cnn_rnn_model], ["SLSTM16", "CNNRNN16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

120
Executing dc3bd026 iter 1
1/1 [==============================] - 0s 22ms/step
Executing 45e525be iter 2
1/1 [==============================] - 0s 21ms/step
Executing 9438dc46 iter 3
1/1 [==============================] - 0s 24ms/step
Executing 0c4f1411 iter 4
1/1 [==============================] - 0s 25ms/step
Executing 4c12e921 iter 5
1/1 [==============================] - 0s 15ms/step
Executing 5451e645 iter 6
1/1 [==============================] - 0s 19ms/step
Executing 25721666 iter 7
1/1 [==============================] - 0s 15ms/step
Executing e5b7a1ea iter 8
1/1 [==============================] - 0s 15ms/step
Executing 8f359b05 iter 9
1/1 [==============================] - 0s 15ms/step
Executing c7eef1fb iter 10
1/1 [==============================] - 0s 15ms/step
Executing 1aa5b59a iter 11
1/1 [==============================] - 0s 14ms/step
Executing 5faf1466 iter 12
1/1 [==============================] - 0s 18ms/step
Executing 4abe4b4a iter 13
1/1 [=========================

# Chile 2Monthly Prediction

In [7]:
region="chile"

In [8]:
metadata = pd.read_csv(f"data/new_features_t2m/{region}/metadata.csv")
metadata = metadata[metadata["frequency"]==FREQUENCY]
metadata.reset_index(drop=True,inplace=True)
metadata

,id,filename,boxes,top_n,var_thresh,modes,frequency,season,methods,extra_indices,only_sea
0,676a31ca,predictor_676a31ca_1.parquet,0,5,NaN,1,2monthly,1,ps,NaN,False
1,676a31ca,predictor_676a31ca_2.parquet,0,5,NaN,1,2monthly,2,ps,NaN,False
2,676a31ca,predictor_676a31ca_3.parquet,0,5,NaN,1,2monthly,3,ps,NaN,False
3,676a31ca,predictor_676a31ca_4.parquet,0,5,NaN,1,2monthly,4,ps,NaN,False
4,676a31ca,predictor_676a31ca_5.parquet,0,5,NaN,1,2monthly,5,ps,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...
1435,a59026f4,predictor_a59026f4_8.parquet,0,30,0.15,3,2monthly,8,ps,PDO-ONI-SAM,False
1436,a59026f4,predictor_a59026f4_9.parquet,0,30,0.15,3,2monthly,9,ps,PDO-ONI-SAM,False
1437,a59026f4,predictor_a59026f4_10.parquet,0,30,0.15,3,2monthly,10,ps,PDO-ONI-SAM,False
1438,a59026f4,predictor_a59026f4_11.parquet,0,30,0.15,3,2monthly,11,ps,PDO-ONI-SAM,False


In [9]:
# last_id = "a3dcae44"
# index = metadata[metadata["id"]==last_id]
# metadata = metadata.iloc[list(index.index)[-1]+1:]
# metadata

In [10]:
# kernel = RBF(length_scale=1.0) + WhiteKernel(noise_level=1)
# regressors = [LinearRegression(),RandomForestRegressor(random_state=42, n_estimators=5), RandomForestRegressor(random_state=42, n_estimators=10), SVR(kernel='rbf'),
#                 XGBRegressor(random_state=42, n_estimators=10, learning_rate=0.1),XGBRegressor(random_state=42, n_estimators=15, learning_rate=0.1), GaussianProcessRegressor(kernel=kernel, random_state=42, n_restarts_optimizer=10)]
# name_regressors = ["Linear", "RF5", "RF10", "SVR-rbf", "XGB10", "XGB15", "GPR"]
# indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]

In [11]:
k=0
id_experiments = metadata["id"].unique()
indices_of_interest = ["HWN", "HWF", "HWD", "HWM", "HWA"]
print(len(id_experiments))
for id in id_experiments:
    k+=1
    print("Executing",id, "iter", k)
    data = {i: pd.read_parquet(f"data/new_features_t2m/{region}/predictor_{id}_{i}.parquet") for i in range(1,13)}
    # rnn_model = Sequential([
    # SimpleRNN(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # lstm_model = Sequential([
    # LSTM(16, activation="tanh", input_shape=(1, len(data[1].columns) - len(indices_of_interest))),
    # Dropout(0.1),  # Regularization
    # Dense(8, activation="relu"),
    # Dense(len(indices_of_interest))  # Predict 5 indices
    # ])
    # Stacked LSTM
    stacked_lstm = Sequential([
        LSTM(16, activation="tanh", return_sequences=True, input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        LSTM(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    cnn_rnn_model = Sequential([
        Conv1D(16, kernel_size=1, activation="relu", input_shape=(1, len(data[12].columns) - len(indices_of_interest))),
        Reshape((1, 16)),  # Back to time dimension
        SimpleRNN(8, activation="tanh"),
        Dropout(0.1),
        Dense(len(indices_of_interest))
    ])
    experiment_1 = PredictionExperiment(data, indices_of_interest, [stacked_lstm, cnn_rnn_model], ["SLSTM16", "CNNRNN16"], 5, id)
    experiment_1.execute_experiment()
    experiment_1.get_metrics("r2", "prediction", show=False)
    experiment_1.get_metrics("mape", "prediction", show=False)
    experiment_1.get_metrics("r2", stage="training", show=False)
    experiment_1.get_metrics("mape", stage="training", show=False)
    experiment_1.get_metrics("r2", stage="TSCV", show=False)
    experiment_1.get_metrics("mape", stage="TSCV", show=False)
    #experiment_1.top_results("r2", 5, stage="prediction", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    #experiment_1.top_results("cv_r2", 5, stage="CV", top_data_path=f"data/results/{FREQUENCY}/top_results.csv")
    experiment_1.save_results(f"data/{region}_results/results.csv")

120
Executing 676a31ca iter 1
1/1 [==============================] - 0s 15ms/step
Executing 20d3f2f7 iter 2
1/1 [==============================] - 0s 15ms/step
Executing e0879131 iter 3
1/1 [==============================] - 0s 17ms/step
Executing f2229bf6 iter 4
1/1 [==============================] - 0s 15ms/step
Executing 864384ff iter 5
1/1 [==============================] - 0s 15ms/step
Executing 5d3808e7 iter 6
1/1 [==============================] - 0s 16ms/step
Executing 1bd170ed iter 7
1/1 [==============================] - 0s 17ms/step
Executing b649f2d0 iter 8
1/1 [==============================] - 0s 17ms/step
Executing 973780e5 iter 9
1/1 [==============================] - 0s 15ms/step
Executing ef86206a iter 10
1/1 [==============================] - 0s 16ms/step
Executing 51d34cae iter 11
1/1 [==============================] - 0s 16ms/step
Executing a9c48acf iter 12
1/1 [==============================] - 0s 18ms/step
Executing bb626a58 iter 13
1/1 [=========================